In [5]:
import numpy as np
from collections import defaultdict
from itertools import islice
import re

In [2]:
# 1. 데이터셋 (간단 예시)
texts = [
    "Box box box",
    "My tyres are gone",
    "Tell him to get out",
    "We need to push",
    "Engine is overheating",
    "Good job keep pushing",
    "I'm losing power",
    "Switching to plan B",
    "Push now push now",
    "Let me race please"
]
labels = [
    "진입 명령", "차량 상태", "불만/요청", "전략/전술", "차량 상태",
    "격려", "차량 상태", "전략/전술", "전략/전술", "불만/요청"
]

In [6]:
def generate_ngrams(text, n=2):
    """Generate n-grams from a given text."""
    tokens = re.findall(r'\b\w+\b', text.lower())
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

In [7]:
def build_vocab(texts, ngram_range=(1, 2)):
    """Build vocabulary for n-grams."""
    vocab = defaultdict(int)
    for text in texts:
        for n in range(ngram_range[0], ngram_range[1] + 1):
            ngrams = generate_ngrams(text, n)
            for ngram in ngrams:
                vocab[ngram] += 1
    return {word: idx for idx, word in enumerate(vocab.keys())}

In [8]:
def vectorize_texts(texts, vocab, ngram_range=(1, 2)):
    """Convert texts to vectorized form using the vocabulary."""
    vectors = []
    for text in texts:
        vec = np.zeros(len(vocab))
        for n in range(ngram_range[0], ngram_range[1] + 1):
            ngrams = generate_ngrams(text, n)
            for ngram in ngrams:
                if ngram in vocab:
                    vec[vocab[ngram]] += 1
        vectors.append(vec)
    return np.array(vectors)

In [9]:
# Build vocabulary and vectorize texts
vocab = build_vocab(texts, ngram_range=(1, 2))
X = vectorize_texts(texts, vocab, ngram_range=(1, 2))

# Encode labels
label2idx = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label2idx[label] for label in labels])
num_classes = len(label2idx)

In [10]:
def train_test_split(X, y, test_size=0.3, seed=42):
    np.random.seed(seed)
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    split = int(len(X) * (1 - test_size))
    train_idx, test_idx = indices[:split], indices[split:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def cross_entropy(probs, y_true):
    batch_size = probs.shape[0]
    log_probs = np.log(probs + 1e-9)
    loss = -log_probs[range(batch_size), y_true].mean()
    return loss

In [12]:
input_dim = X_train.shape[1]
W = np.random.randn(input_dim, num_classes) * 0.01
b = np.zeros((1, num_classes))
learning_rate = 0.1


In [17]:
epochs = 100
for epoch in range(epochs):
    logits = np.dot(X_train, W) + b
    probs = softmax(logits)
    
    loss = cross_entropy(probs, y_train)

    # 수동 gradient 계산
    N = X_train.shape[0]
    one_hot = np.zeros_like(probs)
    one_hot[np.arange(N), y_train] = 1
    dL_dz = (probs - one_hot) / N  # softmax + cross entropy gradient

    # Weight & bias gradients
    dW = np.dot(X_train.T, dL_dz)
    db = np.sum(dL_dz, axis=0, keepdims=True)

    # 경사하강법 업데이트
    W -= learning_rate * dW
    b -= learning_rate * db

    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss:.4f}")

Epoch 0: Loss = 0.0383
Epoch 10: Loss = 0.0363
Epoch 20: Loss = 0.0345
Epoch 30: Loss = 0.0329
Epoch 40: Loss = 0.0314
Epoch 50: Loss = 0.0300
Epoch 60: Loss = 0.0287
Epoch 70: Loss = 0.0276
Epoch 80: Loss = 0.0265
Epoch 90: Loss = 0.0255


In [18]:
test_logits = np.dot(X_test, W) + b
test_probs = softmax(test_logits)
preds = np.argmax(test_probs, axis=1)
acc = np.mean(preds == y_test)

print("\n테스트 정확도:", acc)


테스트 정확도: 0.3333333333333333


In [19]:
# 새로운 데이터 예측
new_texts = [
    "Push harder now",  # 예제 문장
    "Engine is failing"
]

# 1. N-gram 벡터화
new_X_data = []
for text in new_texts:
    vec = np.zeros(len(vocab))
    for n in range(1, 3):  # n-gram 범위 (1, 2)
        ngrams = generate_ngrams(text, n)
        for ngram in ngrams:
            if ngram in vocab:
                vec[vocab[ngram]] += 1
    new_X_data.append(vec)

new_X = np.array(new_X_data)

# 2. 예측
new_logits = np.dot(new_X, W) + b
new_probs = softmax(new_logits)
new_y_pred = np.argmax(new_probs, axis=1)

# 3. 예측 결과를 레이블로 변환
idx2label = {v: k for k, v in label2idx.items()}
new_y_pred_labels = [idx2label[idx] for idx in new_y_pred]

# 4. 결과 출력
for text, label in zip(new_texts, new_y_pred_labels):
    print(f"Text: \"{text}\" | Predicted Label: \"{label}\"")

Text: "Push harder now" | Predicted Label: "전략/전술"
Text: "Engine is failing" | Predicted Label: "불만/요청"
